In [21]:
import pandas as pd
import numpy as np
from collections import OrderedDict
from random import random
import xgboost as xgb
from sklearn.metrics import fbeta_score

In [3]:
# Parameters that are kept constant during the tuning process
param = {'silent':1,
         'min_child_weight':1,
         'objective':'binary:logistic',
         'eval_metric':'auc',
         'seed': 42}

In [2]:
# Parameter search space
tune_dic = OrderedDict()
tune_dic['max_depth']= [5,10,15,20,25] ## maximum tree depth
tune_dic['subsample']=[0.5,0.6,0.7,0.8,0.9,1.0] ## proportion of training instances used in trees
tune_dic['colsample_bytree']= [0.5,0.6,0.7,0.8,0.9,1.0] ## subsample ratio of columns
tune_dic['eta']= [0.01,0.05,0.10,0.20,0.30,0.40]  ## learning rate
tune_dic['gamma']= [0.00,0.05,0.10,0.15,0.20]  ## minimum loss function reduction required for a split
tune_dic['scale_pos_weight']=[1,2,5,8,10,15,20] ## relative weight of positive/negative instances

In [5]:
# Custom metric calculation function
def f2_score(y_pred, y_true): return fbeta_score(y_true, (y_pred>=0.5).astype(int), beta=2)

# Function to train model
def train_model(curr_params, param, Xtrain, Xvalid, num_rounds=20):
    """
    Train the model with given set of hyperparameters
    curr_params - Dict of hyperparameters and chosen values
    param - Dict of hyperparameters that are not tuned
    Xtrain - DMatrix of traing data
    Ytrain - Training labels
    Ytrain - DMatrix of validation data
    Yvalid - Validaion labels
    """
    param.update(curr_params)
    model = xgb.train(param, Xtrain, num_boost_round=num_rounds)
    preds = model.predict(Xvalid)
    labels = Xvalid.get_label()
    f_score = f2_score(preds, labels)
    
    return model, f_score

In [6]:
def choose_params(tune_dic, curr_params=None):
    """
    Function to choose parameters for next iteration, given current parameters
    tune_dic - Dict of Hyperparameter search space
    curr_params - Dict of current hyperparameters
    """
    if curr_params:
        next_params = curr_params.copy()
        param_to_update = np.random.choice(list(tune_dic.keys()))
        param_vals = tune_dic[param_to_update]
        curr_index = param_vals.index(curr_params[param_to_update])
        if curr_index == 0:
            next_params[param_to_update] = param_vals[1]
        elif curr_index == len(param_vals) - 1:
            next_params[param_to_update] = param_vals[curr_index - 1]
        else:
            next_params[param_to_update] = \
                param_vals[curr_index + np.random.choice([-1,1])]
    else:
        next_params = dict()
        for k, v in tune_dic.items():
            next_params[k] = np.random.choice(v)

    return next_params

In [22]:
def simulate_annealing(fn_train, tune_dic, X_train, X_valid, Y_train=None,
                       Y_valid=None, maxiters=100, alpha=0.85, beta=1.3,
                       T=0.40, update_iters=5):
    """
    Function to perform hyperparameter search using simulated annealing
    fn_train - Function to train the model (Should return model and metric value)
    tune_dic - Dictionary of Hyperparameter search space
    maxiters - Number of iterations to perform the parameter search
    alpha - factor to reduce temperature
    beta - constant in probability estimate
    T - Initial temperature
    update_iters - # of iterations required to update temperature
    """
    columns = [*tune_dic.keys()] + ['Metric', 'Best Metric']
    results = pd.DataFrame(index=range(maxiters), columns=columns)
    best_metric = -1.
    prev_metric = -1.
    prev_params = None
    best_params = dict()
    weights = list(map(lambda x: 10**x, list(range(len(tune_dic)))))
    hash_values = set()
    
    for i in range(maxiters):
        print('Starting Iteration {}'.format(i))
        while True:
            curr_params = choose_params(tune_dic, prev_params)
            indices = [tune_dic[k].index(v) for k, v in curr_params.items()]
            hash_val = sum([i * j for (i, j) in zip(weights, indices)])
            if hash_val in hash_values:
                print('Combination revisited')
            else:
                hash_values.add(hash_val)
                break
        
        model, metric = fn_train(
            curr_params,
            param,
            X_train,
            X_valid,
            num_rounds=num_rounds)

        if metric > prev_metric:
            print('Local Improvement from {:8.4f} to {:8.4f} - parameters accepted'\
                  .format(prev_metric, metric))
            prev_params = curr_params.copy()
            prev_metric = metric

            if fscore > best_fscore:
                print('Global improvement from {:8.4f} to {:8.4f} - best parameters updated'\
                  .format(best_metric, metric))
                best_metric = metric
                best_params = curr_params.copy()
        else:
            rnd = np.random.uniform()
            diff = metric - prev_metric
            threshold = np.exp(beta * diff / T)
            if rnd < threshold:
                print("""No Improvement but parameters accepted. F-Score change: {:8.4f} 
                threshold: {:6.4f} random number: {:6.4f}
                """.format(diff, threshold, rnd))
                prev_metric = metric
                prev_params = curr_params
            else:
                print("""No Improvement and parameters rejected. F-Score change: {:8.4f} 
                threshold: {:6.4f} random number: {:6.4f}
                """.format(diff, threshold, rnd))

        results.loc[i, list(curr_params.keys())] = list(curr_params.values())
        results.loc[i, 'F-Score'] = metric
        results.loc[i, 'Best F-Score'] = best_metric

        if i % update_iters == 0: T = alpha * T
        
    return results